# Models and Visualization

This notebook demonstrates the core estimation models in `regimes` (OLS, AR) with structural breaks,
and the full suite of visualization functions.

**Topics covered:**
- OLS with known break points
- AR models with common and variable-specific breaks
- Fit-by-regime analysis
- Break plots, regime shading, confidence intervals
- Parameter comparison plots
- PcGive-style diagnostics

## Setup

In [ ]:
import sys
from pathlib import Path

# Ensure the package is importable when running from the examples/ directory
sys.path.insert(0, str(Path("..") / "src"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regimes as rg

print(f"regimes version: {rg.__version__}")

## Generate Sample Data

In [ ]:
np.random.seed(42)
n = 200
break_point = 100

# Generate X (regressor)
X = np.random.randn(n)

# Generate y with different coefficients before/after break
y = np.zeros(n)
y[:break_point] = 1.0 + 0.5 * X[:break_point] + np.random.randn(break_point) * 0.5
y[break_point:] = 3.0 + 1.5 * X[break_point:] + np.random.randn(n - break_point) * 0.5

# Add constant to X for OLS
X_with_const = np.column_stack([np.ones(n), X])

# Plot the data
plt.figure(figsize=(10, 4))
plt.plot(y)
plt.axvline(x=break_point, color='r', linestyle='--', label=f'True break at {break_point}')
plt.xlabel('Observation')
plt.ylabel('y')
plt.title('Simulated Data with Structural Break')
plt.legend()
plt.show()

## OLS Models with Structural Breaks

### 2.1 OLS Without Breaks (Baseline)

Standard OLS ignoring the structural break. This pools all observations and provides a single set of parameter estimates.

In [ ]:
ols_model = rg.OLS(y, X_with_const)
ols_results = ols_model.fit()
print(ols_results.summary())

### 2.2 OLS With One Known Break

Specify a known break at t=100. The model estimates regime-specific parameters using dummy interactions.

In [ ]:
ols_model_one_break = rg.OLS(y, X_with_const, breaks=[break_point])
ols_results_one_break = ols_model_one_break.fit()
print(ols_results_one_break.summary())

### 2.3 OLS With Two Known Breaks

Multiple breaks can be specified. This creates three regimes with separate parameter estimates.

In [ ]:
ols_model_two_breaks = rg.OLS(y, X_with_const, breaks=[100, 150])
ols_results_two_breaks = ols_model_two_breaks.fit()
print(ols_results_two_breaks.summary())

### 2.4 OLS Fit by Regime

`fit_by_regime()` estimates separate OLS models for each regime, allowing completely independent specifications. Returns a list of OLSResults objects.

In [ ]:
ols_model_full_break = rg.OLS(y, X_with_const, breaks=[break_point])
ols_results_full_break = ols_model_full_break.fit_by_regime()

print("Regime 1 (before break):")
print(ols_results_full_break[0].summary())
print("\nRegime 2 (after break):")
print(ols_results_full_break[1].summary())

## AR Models with Structural Breaks

### Generate AR Data

In [ ]:
np.random.seed(123)
n_ar = 200
break_ar = 100
phi = 0.7  # AR coefficient (stable across regimes)

y_ar = np.zeros(n_ar)
for t in range(1, n_ar):
    c = 1.0 if t < break_ar else 3.0
    y_ar[t] = c + phi * y_ar[t-1] + np.random.randn()

# Plot the simulated series
plt.figure(figsize=(10, 4))
plt.plot(y_ar)
plt.axvline(x=break_ar, color='r', linestyle='--', label=f'True break at {break_ar}')
plt.xlabel('Observation')
plt.ylabel('y')
plt.title('AR(1) with Intercept Shift (stable AR coefficient)')
plt.legend()
plt.show()

### 3.2 AR Without Breaks (Baseline)

Standard AR(1) model ignoring the structural break.

In [ ]:
ar_constant = rg.AR(y_ar, lags=1)
ar_results_constant = ar_constant.fit()
print(ar_results_constant.summary())

### 3.3 AR With Common Breaks (All Coefficients Change)

Use `breaks=` to allow all parameters (constant and AR coefficients) to change at the break.

In [ ]:
ar_common = rg.AR(y_ar, lags=1, breaks=[break_ar])
results_common = ar_common.fit()
print(results_common.summary())

### 3.4 AR With Variable-Specific Breaks (Only Intercept Changes)

Use `variable_breaks=` to specify which parameters can change. Here only the constant shifts, while the AR coefficient is constrained to be the same across regimes.

In [ ]:
ar_variable = rg.AR(y_ar, lags=1, variable_breaks={"const": [break_ar]})
results_variable = ar_variable.fit()
print(results_variable.summary())

### 3.5 AR Fit by Regime

Fit completely separate AR models for each regime. Returns a list of ARResults objects.

In [ ]:
ar_regime = rg.AR(y_ar, lags=1, breaks=[break_ar])
results_by_regime = ar_regime.fit_by_regime()
print(rg.ar_summary_by_regime(results_by_regime, breaks=[break_ar], nobs_total=n_ar))

## Visualization

First, run Bai-Perron to get break results for the visualization demos:

In [ ]:
bp_mean_shift = rg.BaiPerronTest(y)
bp_results_mean = bp_mean_shift.fit()
print(f"Detected breaks: {bp_results_mean.break_indices}")
print(bp_results_mean.summary())

In [ ]:
bp_results = bp_mean_shift

### 5.1 `plot_breaks()` â€” Basic Usage

Plot time series with vertical lines at break dates.

In [ ]:
fig, ax = rg.plot_breaks(y, breaks=[100])
plt.show()

### 5.2 `plot_breaks()` â€” With `results` Parameter

Pass Bai-Perron results directly to extract breaks automatically.

In [ ]:
fig, ax = rg.plot_breaks(y, results=bp_results)
plt.show()

### 5.3 `plot_breaks()` â€” With Regime Shading

Set `shade_regimes=True` to color-code different regimes.

In [ ]:
fig, ax = rg.plot_breaks(y, results=bp_results, shade_regimes=True)
plt.show()

### 5.4 `plot_regime_means()` â€” Regime Means Overlay

Plot the data with horizontal lines showing the mean in each regime.

In [ ]:
fig, ax = rg.plot_regime_means(y, breaks=bp_results.break_indices)
plt.show()

### 5.5 `plot_break_confidence()` â€” Confidence Intervals

Visualize uncertainty in break date estimation with confidence bands.

In [ ]:
# Get confidence intervals from Bai-Perron results (or define manually)
confidence_intervals = [(b - 10, b + 10) for b in bp_results.break_indices]
fig, ax = rg.plot_break_confidence(
    y,
    breaks=bp_results.break_indices,
    confidence_intervals=confidence_intervals,
    title="Detected Breaks with Confidence Intervals"
)
plt.show()

### 5.6 `plot_params_over_time()` â€” Single Model

Visualize how parameter estimates change across regimes as step functions.

In [ ]:
fig, axes = rg.plot_params_over_time(
    ols_results_one_break,
    figsize=(10, 5),
    title="OLS Parameter Estimates with One Break"
)
plt.tight_layout()
plt.show()

### 5.7 `plot_params_over_time()` â€” Comparing Multiple Models

Pass a dictionary of results to compare models with different break assumptions side by side.

In [ ]:
fig, axes = rg.plot_params_over_time(
    {
        "No breaks": ols_results,
        "Known break at 100": ols_results_one_break,
        "Bai-Perron detected": ols_with_breaks,
    },
    figsize=(12, 6),
    title="Comparing OLS Parameter Estimates Across Model Specifications"
)
plt.tight_layout()
plt.show()

### 5.8 `plot_diagnostics()` â€” PcGive-Style Misspecification Diagnostics

The `plot_diagnostics()` function creates a 2Ã—2 panel of diagnostic plots mimicking OxMetrics/PcGive:
- **Actual vs Fitted**: Time series comparison
- **Residual Distribution**: Histogram with N(0,1) overlay
- **Scaled Residuals**: Vertical index plot (resid/Ïƒ) for spotting autocorrelation
- **ACF/PACF**: Autocorrelation and partial autocorrelation functions

In [ ]:
# Method access on results object
fig, axes = ols_results.plot_diagnostics()
plt.show()

### 5.9 Individual Diagnostic Plots

Each diagnostic component is also available as a standalone function for customization.

In [ ]:
# Individual diagnostic functions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Actual vs Fitted
rg.plot_actual_fitted(ols_results, ax=axes[0, 0])

# Residual Distribution
rg.plot_residual_distribution(ols_results, ax=axes[0, 1])

# Scaled Residuals
rg.plot_scaled_residuals(ols_results, ax=axes[1, 0])

# ACF/PACF needs its own subplots
axes[1, 1].remove()  # Remove the placeholder
fig_acf, ax_acf = rg.plot_residual_acf(ols_results, nlags=15)

plt.tight_layout()
plt.show()

### 5.10 Diagnostics for AR and ADL Models

Diagnostic plots work with all regression model types (OLS, AR, ADL).

In [ ]:
# Diagnostics for AR model
fig, axes = ar_results_constant.plot_diagnostics(nlags=15)
plt.show()